In [50]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
import string
from nltk import pos_tag
from nltk.corpus import stopwords
# nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from nltk.corpus import wordnet
import re
from preprocessing import *
from numpy import array 
import numpy as np

In [51]:
w2v_pretrained = Word2Vec.load("w2vPreTrained")

In [52]:
dataset = pd.read_csv("amazon_alexa.tsv", sep = "\t", encoding = "utf-8")
print(dataset.shape)
dataset.dropna(inplace = True)
print(dataset.shape)
dataset.drop(dataset[dataset.rating == 3].index, inplace=True)
print(dataset.shape)
dataset.drop_duplicates(subset = "verified_reviews", inplace = True)
print(dataset.shape)

(3150, 6)
(3150, 6)
(2998, 6)
(2196, 6)


In [53]:
X = np.array(dataset["verified_reviews"].values).reshape(-1, 1)
y = list(dataset["feedback"].values)

In [54]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state = 0)

X, y = undersampler.fit_resample(X, y)


print('Resampled dataset shape %s' % Counter(y))

Resampled dataset shape Counter({1: 412, 0: 206})


In [55]:
X_temp = []

for rev in X:
  X_temp.append(rev[0])

In [56]:
tokenized_reviews, sentences = tokenize_list_of_text(X_temp, custom_stopwords, False,2)

total number of types extracted is: 1788


### *Count Vectorizer:*
grazie ad esso è stata ottenuta una document-term matrix popolata dalla frequenza assoluta dei token (con frequenza minima due) 

In [57]:
tokenizer = nltk.tokenize.TweetTokenizer() #
cv = CountVectorizer(stop_words="english", ngram_range=(1, 3), tokenizer=tokenizer.tokenize, min_df = 2) #count==freq
text_counts = cv.fit_transform(tokenized_reviews) 
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, y, test_size = 0.20, random_state=10) 
#divisione in train e test 0.20, rs=seed=10, split=rip

C:\Users\Alice\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [59]:
#otteniamo matrice 412+206
text_counts

<618x1224 sparse matrix of type '<class 'numpy.int64'>'
	with 6987 stored elements in Compressed Sparse Row format>

In [60]:
len(cv.vocabulary_) 

1224

In [61]:
cv.vocabulary_ # Tutti gli id collegati alle parole del vocabolario

{'like': 520,
 'fact': 311,
 'answer': 44,
 'not_see': 680,
 'real': 823,
 'need': 638,
 'household': 452,
 'good': 377,
 'day': 208,
 'deal': 213,
 'sound': 968,
 'terrible': 1032,
 'want': 1153,
 'music': 624,
 'bose': 104,
 'sound terrible': 976,
 'good music': 383,
 'little': 538,
 'feature': 330,
 'stop': 999,
 'work': 1190,
 'week': 1173,
 'command': 160,
 'really': 826,
 'fun': 362,
 'stop work': 1001,
 'work week': 1208,
 'joke': 488,
 'worthless': 1213,
 'disappointed': 248,
 'plug': 756,
 'wall': 1152,
 'time': 1047,
 'fault': 327,
 'disagree': 246,
 'make': 595,
 'company': 163,
 'advance': 28,
 'sell': 910,
 'product': 785,
 'battery': 86,
 'return': 870,
 'apple': 52,
 'speaker': 980,
 'really disappointed': 827,
 'apple music': 53,
 'great': 393,
 'bass': 84,
 'idle': 459,
 'hot': 449,
 'miss': 614,
 'recognize': 837,
 'thing': 1037,
 'great sound': 411,
 'idle time': 461,
 'impressed': 466,
 'tightness': 1046,
 'stupid': 1008,
 'spotify': 989,
 'account': 14,
 'not_use':

In [62]:
#get_feature_names_out([input_features])
#Get output feature names for transformation.

len(cv.get_feature_names_out())

1224

In [63]:
X_train[0,:] #prima recensione

<1x1224 sparse matrix of type '<class 'numpy.int64'>'
	with 18 stored elements in Compressed Sparse Row format>

In [64]:
print(X_train[0,:])

  (0, 520)	1
  (0, 1190)	1
  (0, 1173)	1
  (0, 106)	1
  (0, 550)	1
  (0, 556)	1
  (0, 845)	1
  (0, 644)	1
  (0, 735)	1
  (0, 1204)	1
  (0, 798)	1
  (0, 320)	1
  (0, 422)	2
  (0, 716)	1
  (0, 423)	1
  (0, 524)	1
  (0, 324)	1
  (0, 1174)	1


In [65]:
cv.inverse_transform(X_train[0,:]) 
#0=>1' recensione: xtrain mat, id-parole

[array(['like', 'work', 'week', 'box', 'look', 'look like', 'refurbish',
        'new', 'perfectly', 'work perfectly', 'purchase', 'far', 'happy',
        'original', 'happy purchase', 'like new', 'far work', 'week far'],
       dtype='<U25')]

In [66]:
for feat,freq in zip(cv.inverse_transform(X_train[0,:])[0],X_train[0,:].data):
    print(feat,freq)

like 1
work 1
week 1
box 1
look 1
look like 1
refurbish 1
new 1
perfectly 1
work perfectly 1
purchase 1
far 1
happy 2
original 1
happy purchase 1
like new 1
far work 1
week far 1


### SelectKBest

In [67]:
#For classification we'll set 'chi2'  method as a scoring function. The target number of features is defined by k parameter
select = SelectKBest(chi2, k=800)  # feature selection
# select = SelectKBest(chi2, k="all")  # feature selection for balanced with pos filter
select.fit(X_train,Y_train)
X_train_sel = select.transform(X_train)
X_test_sel = select.transform(X_test)

In [117]:
filter = select.get_support() #Lista filter: filtra le parole contenenute nei k 800
sum(filter), len(filter)

(800, 1224)

In [118]:
X_train_sel

<494x800 sparse matrix of type '<class 'numpy.int64'>'
	with 4270 stored elements in Compressed Sparse Row format>

In [70]:
print(X_test_sel[0,:])

  (0, 773)	1
  (0, 356)	1
  (0, 376)	1


In [71]:
print(cv.inverse_transform(select.inverse_transform(X_train_sel[0,:]))) 
#applichi l'inverse_transform sui vettori utilizzati con il Select

[array(['box', 'far', 'far work', 'happy', 'happy purchase', 'like new',
       'look like', 'new', 'original', 'refurbish', 'week', 'week far',
       'work'], dtype='<U25')]


### TFIDF
è una tecnica di pesatura delle parole 
Il TF indica la frequenza con cui una parola appare in un documento, mentre l'IDF indica la rarità della parola in tutto il corpus dei documenti.


In [72]:
tfidf = TfidfTransformer()  # weighting
tfidf.fit(X_train_sel)
X_train_vec = tfidf.transform(X_train_sel)
X_test_vec =tfidf.transform(X_test_sel)

In [73]:
print(X_train_vec[0,:])

  (0, 773)	0.13215721023034152
  (0, 762)	0.3104364762375252
  (0, 761)	0.2545811907451864
  (0, 539)	0.22860996199910866
  (0, 456)	0.24922448598010277
  (0, 406)	0.1987259052528476
  (0, 351)	0.28446524749144747
  (0, 328)	0.27519571472618043
  (0, 264)	0.28446524749144747
  (0, 263)	0.4717708962714922
  (0, 201)	0.3104364762375252
  (0, 198)	0.1987259052528476
  (0, 72)	0.2673584364439322


In [74]:
cv.inverse_transform(select.inverse_transform(X_train_vec[0,:]))

[array(['box', 'far', 'far work', 'happy', 'happy purchase', 'like new',
        'look like', 'new', 'original', 'refurbish', 'week', 'week far',
        'work'], dtype='<U25')]

In [75]:
for feat,weight,freq in zip(cv.inverse_transform(select.inverse_transform(X_train_vec[0,:]))[0],X_train_vec[0,:].data,X_train_sel[1,:].data):
    print(feat,round(weight,4),freq)

box 0.1322 1
far 0.3104 1
far work 0.2546 1
happy 0.2286 1
happy purchase 0.2492 1
like new 0.1987 1
look like 0.2845 1
new 0.2752 1


In [76]:
# create a dataframe with words, tf_idf score and freq
total_scores = []

for i in range(X_train_vec.shape[0]):
    for feat,weight,freq in zip(cv.inverse_transform(select.inverse_transform(X_train_vec[i,:]))[0],X_train_vec[i,:].data,X_train_sel[i,:].data):
        total_scores.append((feat,round(weight,4),freq))

words = []
tf_idf_score = []
freq = []
for i in range(len(total_scores)):
    words.append(total_scores[i][0])
    tf_idf_score.append(total_scores[i][1])
    freq.append(total_scores[i][2])

scores_df = pd.DataFrame()
scores_df["words"] = words
scores_df["tf_idf_score"] = tf_idf_score
scores_df["freq"] = freq

In [77]:
scores_df.sort_values(by = "tf_idf_score", ascending = False).head(50) # sort and show the df

,words,tf_idf_score,freq
2715,good,1.0000,1
4148,new,1.0000,1
900,not_youtube,1.0000,1
332,not_problem,1.0000,1
4036,idle,1.0000,1
2791,work,1.0000,1
2779,disconnect,1.0000,1
1043,tell,1.0000,1
2748,love,1.0000,1
2738,love,1.0000,1


In [78]:
X_train_vec

<494x800 sparse matrix of type '<class 'numpy.float64'>'
	with 4270 stored elements in Compressed Sparse Row format>

### LSVM
Il processo di addestramento dell'SVM comporta la selezione di un sottoinsieme di campioni di addestramento (noto come vettori di supporto) che si trovano vicino al margine dell'iperpiano. 

In [79]:
svm = LinearSVC()  # linear svm with default parameters
svm_clf = svm.fit(X_train_vec,Y_train)
predictions = svm_clf.predict(X_test_vec)

In [80]:
len(predictions), sum(predictions)

(124, 86)

In [81]:
print(metrics.classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.74      0.68      0.71        41
           1       0.85      0.88      0.86        83

    accuracy                           0.81       124
   macro avg       0.79      0.78      0.79       124
weighted avg       0.81      0.81      0.81       124



In [82]:
feature_names = cv.get_feature_names_out()
feature_names

array(["'", "' ll", "' ve", ..., 'yes', 'zero', 'zigbee'], dtype=object)

In [83]:
print(select.scores_)

[4.47727733 1.99393939 0.24772958 ... 1.99393939 3.98787879 7.97575758]


In [84]:
feature_names = cv.get_feature_names_out()
feats_w_score = list()
for index,(selected,score) in enumerate(zip(filter,select.scores_)):#enumerate ritorna una tupla con la parola e la sua posizione
    feats_w_score.append((score,selected,feature_names[index])) #zip invece cicla più liste in contemporanea.
feats_w_score = sorted(feats_w_score, reverse = True)
len(feats_w_score)

1224

In [85]:
feats_w_score[:10] # this contains k score, wether it is a selcted feature and the word

[(26.1792791786454, True, 'great'),
 (22.24837695232832, True, 'send'),
 (22.24837695232832, True, 'month'),
 (18.399484203739526, True, 'bulb'),
 (13.95757575757576, True, 'actually'),
 (13.450001535107923, True, 'money'),
 (13.44703782400041, True, 'ask'),
 (13.265352056067726, True, 'buy'),
 (11.963636363636367, True, 'bridge'),
 (11.963636363636367, True, 'act')]

In [86]:
type(svm)

sklearn.svm._classes.LinearSVC

In [87]:
len(svm.coef_) #ritorna una tupla con la parola e la sua posizione)

1

In [88]:
feats_w_classifier_weight = list()
for index,weight in enumerate(select.inverse_transform(svm.coef_)[0]):
    if weight!=0:
        feats_w_classifier_weight.append((weight,feature_names[index]))
feats_w_classifier_weight = sorted(feats_w_classifier_weight)
len(feats_w_classifier_weight)
#ritorna per ognuna delle 800 parole, il punteggio svm

786

In [89]:
feats_w_classifier_weight[-100:] #features positive

[(0.36017726498705754, 'internal'),
 (0.36076637185310306, 'love want'),
 (0.36115591468672176, 'love thing'),
 (0.36158627029858775, 'ring'),
 (0.3627215352259915, 'clarity'),
 (0.36382979256893155, 'tell'),
 (0.36613199826584075, 'added'),
 (0.37315004609134667, 'amazing'),
 (0.37747166822522116, 'way use'),
 (0.3781770545805382, 'light'),
 (0.37883605729133885, 'sound quality'),
 (0.3789405785041359, 'microphone'),
 (0.378940578504136, 'okay'),
 (0.3853973898163297, 'set'),
 (0.38598941717763996, 'love buy'),
 (0.38785882277004663, 'new like'),
 (0.38906018570916034, 'ease'),
 (0.38906018570916034, 'ease use'),
 (0.3893977692015775, 'like new'),
 (0.38991518477058895, 'primarily'),
 (0.39433863513323236, 'product'),
 (0.394378158287919, 'bedside'),
 (0.396368464571094, 'good buy'),
 (0.39777146585788015, 'listen music'),
 (0.3978069319463577, 'variety'),
 (0.39855987050829506, 'job'),
 (0.4042074082712679, 'music'),
 (0.4110398881798771, 'small'),
 (0.41165597472604776, 'packaging')

In [90]:
feats_w_classifier_weight[:100] #features negative

[(-1.6567079683162813, 'idle'),
 (-1.2737693928767229, 'terrible'),
 (-1.160728821253476, 'return'),
 (-1.1531558764690448, 'try'),
 (-1.1475947057413731, 'not_see'),
 (-1.0912373215003381, 'pay'),
 (-1.0822484908857628, 'disconnect'),
 (-0.9931745330794509, 'difficult'),
 (-0.9919057979850527, 'useless'),
 (-0.935740624175972, 'disappointed'),
 (-0.8749608902644336, 'speak'),
 (-0.8672293717966648, 'dont'),
 (-0.8474638762229424, 'poor'),
 (-0.8373732993695485, 'switch'),
 (-0.8331723676316034, 'functionality'),
 (-0.8240470670382425, 'cheap'),
 (-0.8194373759927387, 'make'),
 (-0.8074319666259892, 'meh'),
 (-0.8014627997673492, 'obviate'),
 (-0.7928335600585792, 'turn'),
 (-0.7895626080764457, 'poor quality'),
 (-0.7883286487754347, 'spanish'),
 (-0.7823804998359652, 'slow'),
 (-0.7789676442902131, 'worthless'),
 (-0.7789675283626429, 'not_youtube'),
 (-0.7717669115621434, 'hue'),
 (-0.7594926965727561, 'maybe'),
 (-0.7551017085240141, 'sound terrible'),
 (-0.7479586551308297, 'party

In [91]:
df_scores = pd.DataFrame()
scores = []
words = []
for entry in feats_w_classifier_weight:
    scores.append(entry[0])
    words.append(entry[1])

df_scores["scores"] = scores
df_scores["words"] = words

df_scores.sort_values(by = "scores", ascending = False).head(15) # use tail for seeing the positive words

,scores,words
785,1.947914,love
784,1.356124,great
783,1.219154,easy
782,0.930868,search
781,0.877211,good
780,0.806725,far
779,0.771854,best
778,0.769529,lol
777,0.752497,figure
776,0.738027,perfect


Qui testiamo una stringa, la passiamo a cv, a skb, a tfidf e la prediciamo con svm. 1 se positiva, 0 se negativa

In [121]:
stringa = "the product is disappointing, bad at all"
clean = tokenize_list_of_text([stringa])[0]
clean

total number of types extracted is: 5


['the product disappointing bad all']

In [122]:
vector = cv.transform(clean)
vector = select.transform(vector)
vector = tfidf.transform(vector)
predicted = svm.predict(vector)
print(predicted)

[0]


### Multinomial Naive Bayes: 

Algoritmo basato sul teorema di Bayes,che afferma che la probabilità di un'ipotesi H (in questo caso, un'etichetta di classe) data qualche prova E (in questo caso, un insieme di caratteristiche) è proporzionale alla probabilità dell'evidenza data dall'ipotesi moltiplicata per la probabilità precedente dell'ipotesi: P(H | E) = P(E | H) * P(H) / P(E) 
Nel caso di Multinomial Naive Bayes, l'algoritmo assume che le caratteristiche siano tratte da una distribuzione multinomiale, e che la probabilità di ogni caratteristica data l'etichetta della classe sia indipendente dalle altre caratteristiche.

In [111]:
MNB = MultinomialNB()  # MNB with default parameters
MNB_clsf = MNB.fit(X_train_vec,Y_train)
predictions = MNB_clsf.predict(X_test_vec)
print(metrics.classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.90      0.46      0.61        41
           1       0.79      0.98      0.87        83

    accuracy                           0.81       124
   macro avg       0.85      0.72      0.74       124
weighted avg       0.83      0.81      0.79       124



In [95]:
len(Y_test)

124

### SVC
 è stata utilizzata una Grid-Search al fine di selezionare gliiperparametri che massimizzassero l’f1-macro. 
 L’addestramento è stato eseguito tramite la Stratified KFold Cross-validation con 5 fold.

In [123]:
#iteriamo su svc, non lineare
#Kernel, che determina la funzione usata per trasformare i dati in input in uno spazio ad alto numero di dimensioni;
#C, che controlla la quantità di penalizzazione assegnata ad ev errori di classif nel processo di add del modello;
#Gamma,  controlla l’influenza di ogni esempio del training;
#Class weight, parametro di regolazione che viene utilizzato per gestire le classi sbilanciate.

In [112]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf','linear', 'poly'],
              'class_weight':['balanced', None]
}

grid = GridSearchCV(SVC(), param_grid, refit ='f1_macro', scoring="f1_macro", cv=kfold, verbose=3)
grid.fit(X_train_vec,Y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.739 total time=   0.0s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.910 total time=   0.0s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.792 total time=   0.0s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.716 total time=   0.0s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.363 total time=   0.0s
[CV 1/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.687 total time=   0.0s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.677 total time=   0.0s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.744 total time=   0.0s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.646 total time=   0.0s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=1, kernel=lin

[CV 5/5] END C=0.1, class_weight=None, gamma=0.1, kernel=poly;, score=0.399 total time=   0.0s
[CV 1/5] END C=0.1, class_weight=None, gamma=0.01, kernel=rbf;, score=0.400 total time=   0.0s
[CV 2/5] END C=0.1, class_weight=None, gamma=0.01, kernel=rbf;, score=0.400 total time=   0.0s
[CV 3/5] END C=0.1, class_weight=None, gamma=0.01, kernel=rbf;, score=0.400 total time=   0.0s
[CV 4/5] END C=0.1, class_weight=None, gamma=0.01, kernel=rbf;, score=0.400 total time=   0.0s
[CV 5/5] END C=0.1, class_weight=None, gamma=0.01, kernel=rbf;, score=0.399 total time=   0.0s
[CV 1/5] END C=0.1, class_weight=None, gamma=0.01, kernel=linear;, score=0.400 total time=   0.0s
[CV 2/5] END C=0.1, class_weight=None, gamma=0.01, kernel=linear;, score=0.400 total time=   0.0s
[CV 3/5] END C=0.1, class_weight=None, gamma=0.01, kernel=linear;, score=0.400 total time=   0.0s
[CV 4/5] END C=0.1, class_weight=None, gamma=0.01, kernel=linear;, score=0.400 total time=   0.0s
[CV 5/5] END C=0.1, class_weight=None,

[CV 5/5] END C=1, class_weight=balanced, gamma=0.001, kernel=poly;, score=0.318 total time=   0.0s
[CV 1/5] END C=1, class_weight=None, gamma=1, kernel=rbf;, score=0.843 total time=   0.0s
[CV 2/5] END C=1, class_weight=None, gamma=1, kernel=rbf;, score=0.836 total time=   0.0s
[CV 3/5] END C=1, class_weight=None, gamma=1, kernel=rbf;, score=0.771 total time=   0.0s
[CV 4/5] END C=1, class_weight=None, gamma=1, kernel=rbf;, score=0.833 total time=   0.0s
[CV 5/5] END C=1, class_weight=None, gamma=1, kernel=rbf;, score=0.825 total time=   0.0s
[CV 1/5] END C=1, class_weight=None, gamma=1, kernel=linear;, score=0.895 total time=   0.0s
[CV 2/5] END C=1, class_weight=None, gamma=1, kernel=linear;, score=0.838 total time=   0.0s
[CV 3/5] END C=1, class_weight=None, gamma=1, kernel=linear;, score=0.756 total time=   0.0s
[CV 4/5] END C=1, class_weight=None, gamma=1, kernel=linear;, score=0.953 total time=   0.0s
[CV 5/5] END C=1, class_weight=None, gamma=1, kernel=linear;, score=0.858 total

[CV 5/5] END C=10, class_weight=balanced, gamma=0.01, kernel=rbf;, score=0.799 total time=   0.0s
[CV 1/5] END C=10, class_weight=balanced, gamma=0.01, kernel=linear;, score=0.879 total time=   0.0s
[CV 2/5] END C=10, class_weight=balanced, gamma=0.01, kernel=linear;, score=0.838 total time=   0.0s
[CV 3/5] END C=10, class_weight=balanced, gamma=0.01, kernel=linear;, score=0.766 total time=   0.0s
[CV 4/5] END C=10, class_weight=balanced, gamma=0.01, kernel=linear;, score=0.847 total time=   0.0s
[CV 5/5] END C=10, class_weight=balanced, gamma=0.01, kernel=linear;, score=0.793 total time=   0.0s
[CV 1/5] END C=10, class_weight=balanced, gamma=0.01, kernel=poly;, score=0.400 total time=   0.0s
[CV 2/5] END C=10, class_weight=balanced, gamma=0.01, kernel=poly;, score=0.400 total time=   0.0s
[CV 3/5] END C=10, class_weight=balanced, gamma=0.01, kernel=poly;, score=0.400 total time=   0.0s
[CV 4/5] END C=10, class_weight=balanced, gamma=0.01, kernel=poly;, score=0.400 total time=   0.0s
[

[CV 1/5] END C=100, class_weight=balanced, gamma=1, kernel=linear;, score=0.851 total time=   0.0s
[CV 2/5] END C=100, class_weight=balanced, gamma=1, kernel=linear;, score=0.839 total time=   0.0s
[CV 3/5] END C=100, class_weight=balanced, gamma=1, kernel=linear;, score=0.761 total time=   0.0s
[CV 4/5] END C=100, class_weight=balanced, gamma=1, kernel=linear;, score=0.847 total time=   0.0s
[CV 5/5] END C=100, class_weight=balanced, gamma=1, kernel=linear;, score=0.805 total time=   0.0s
[CV 1/5] END C=100, class_weight=balanced, gamma=1, kernel=poly;, score=0.480 total time=   0.0s
[CV 2/5] END C=100, class_weight=balanced, gamma=1, kernel=poly;, score=0.667 total time=   0.0s
[CV 3/5] END C=100, class_weight=balanced, gamma=1, kernel=poly;, score=0.432 total time=   0.0s
[CV 4/5] END C=100, class_weight=balanced, gamma=1, kernel=poly;, score=0.518 total time=   0.0s
[CV 5/5] END C=100, class_weight=balanced, gamma=1, kernel=poly;, score=0.509 total time=   0.0s
[CV 1/5] END C=100, 

[CV 2/5] END C=100, class_weight=None, gamma=0.01, kernel=linear;, score=0.801 total time=   0.0s
[CV 3/5] END C=100, class_weight=None, gamma=0.01, kernel=linear;, score=0.739 total time=   0.0s
[CV 4/5] END C=100, class_weight=None, gamma=0.01, kernel=linear;, score=0.837 total time=   0.0s
[CV 5/5] END C=100, class_weight=None, gamma=0.01, kernel=linear;, score=0.832 total time=   0.0s
[CV 1/5] END C=100, class_weight=None, gamma=0.01, kernel=poly;, score=0.400 total time=   0.0s
[CV 2/5] END C=100, class_weight=None, gamma=0.01, kernel=poly;, score=0.400 total time=   0.0s
[CV 3/5] END C=100, class_weight=None, gamma=0.01, kernel=poly;, score=0.400 total time=   0.0s
[CV 4/5] END C=100, class_weight=None, gamma=0.01, kernel=poly;, score=0.400 total time=   0.0s
[CV 5/5] END C=100, class_weight=None, gamma=0.01, kernel=poly;, score=0.399 total time=   0.0s
[CV 1/5] END C=100, class_weight=None, gamma=0.001, kernel=rbf;, score=0.432 total time=   0.0s
[CV 2/5] END C=100, class_weight

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'class_weight': ['balanced', None],
                         'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'linear', 'poly']},
             refit='f1_macro', scoring='f1_macro', verbose=3)

In [113]:
print(grid.best_params_)

{'C': 1, 'class_weight': None, 'gamma': 1, 'kernel': 'linear'}


In [114]:
grid_predictions = grid.predict(X_test_vec)

print(metrics.classification_report(Y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.74      0.68      0.71        41
           1       0.85      0.88      0.86        83

    accuracy                           0.81       124
   macro avg       0.79      0.78      0.79       124
weighted avg       0.81      0.81      0.81       124

